In [1]:
# Import packages

import pandas as pd 
import numpy as np
import re

import nltk
from nltk import word_tokenize as w_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer as tfv
from sklearn.decomposition import NMF as nmf

import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.high_level import extract_text
import PyPDF2
# !pip install PyPDF2

In [3]:
# extract text from list of pdf titles and create dataframe of text

path = './Scripts/'
end = '.pdf'
titles = ['1917', '20th_Century_Women', 'BridgeofSpies', 'ExMachina', 'First_Reformed',
         'Get_Out', 'GreenBook', 'Hell_or_HighWater', 'InsideOut', 'KnivesOut', 'LadyBird',
         'LaLaLand', 'Manchester_By_TheSea', 'MarriageStory', 'Parasite', 'Roma', 
          'ShapeofWater', 'Spotlight', 'StraightOuttaCompton', 'TheBigSick', 
         'TheFavourite', 'TheLobster', 'ThreeBillboards', 'Vice']

movie_scripts = []
for title in titles:
    script = extract_text(path + title + end)
    movie_scripts.append(script)
    print (title)

# create dataframe of scripts/titles from lists    
movie_df = pd.DataFrame(list(zip(titles, movie_scripts)), columns = ['Title', 'Script'])

1917
20th_Century_Women
BridgeofSpies
ExMachina
First_Reformed
Get_Out
GreenBook
Hell_or_HighWater
InsideOut
KnivesOut
LadyBird
LaLaLand
Manchester_By_TheSea
MarriageStory
Parasite
Roma
ShapeofWater
Spotlight
StraightOuttaCompton
TheBigSick
TheFavourite
TheLobster
ThreeBillboards
Vice


In [13]:
# add label for original screenplay win, best picture win, and year
orig_screen_win = [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
best_pic_win = ['Nom', None, 'Nom', None, None, 'Nom', 'Win', 'Nom', None, None, 'Nom', 'Nom', 'Nom', 'Nom', 'Win', 'Nom', 'Win', 'Win', None, None, 'Nom', None, 'Nom', 'Nom']
year = [2019, 2016, 2015, 2015, 2018, 2017, 2018, 2016, 2015, 2019, 2017, 2016, 2016, 2019, 2019, 2018, 2017, 2015, 2015, 2017, 2018, 2016, 2017, 2018]
reviewer = ["Shafer", "Shafer", "Shafer", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Shafer", "Ragan", "Ragan", "Shafer", "Shafer", "Ragan", "Ragan", "Shafer", "Ragan", "Ragan"]

# ADD GENRE

# sanity check -- should return True
len(orig_screen_win) == len(year) == len(best_pic_win) ==  len(list(movie_df['Title'])) == len(reviewer)

movie_df['year'] = year
movie_df['orig_screen_win'] = orig_screen_win
movie_df['best_pic_win'] = best_pic_win
movie_df['reviewer'] = reviewer

True

In [12]:
movie_df

# column of dialogue
# column of visual cues
# column of setting
# raw script column
# everything column


# corpus of just dialogue
# corpus of everything (dialogue, settings, visual cues, etc.) cleaned up
# corpus of visual cues

# so 3 x per model
    # sentiment analysis on dialogue (CV + TFIDF)
    # sentiment analysis on everything (CV + TFIDF)
    # sentimental analysis on visual cues
    # and so on forth

# POS tagging and keep only proper names and put rest into settings (when dialogue vs setting can both be caps) 

,Title,Script,year,orig_screen_win,best_pic_win,reviewer
0,1917,1917\n\nWritten by\n\nSam Mendes\n&\nKrysty Wi...,2019,0,Nom,Shafer
1,20th_Century_Women,20th Century Women\n\nby\n\nMike Mills\n\n EXT...,2016,0,None,Shafer
2,BridgeofSpies,Scripts.com\n\nBridge of Spies\n\nBy Matt Char...,2015,0,Nom,Shafer
3,ExMachina,\n\nEX MACHINA\n\nBy \n\nAlex Garland\n\n© Al...,2015,0,None,Shafer
4,First_Reformed,Scripts.com\n\nFirst Reformed\n\nBy Paul Schra...,2018,0,None,Ragan
5,Get_Out,Scripts.com\n\nGet Out\n\nBy Jordan Peele\n\nP...,2017,1,Nom,Shafer
6,GreenBook,GREEN BOOK \n\nWritten by \n\nNick Vallelonga ...,2018,1,Win,Ragan
7,Hell_or_HighWater,Scripts.com\n\nHell or High Water\n\nBy Taylor...,2016,0,Nom,Ragan
8,InsideOut,INSIDE OUT\n\noriginal story by\n\nPete Docter...,2015,0,None,Shafer
9,KnivesOut,KNIVES OUT\n\nA Murder Mystery by\n\nRian John...,2019,0,None,Shafer


In [ ]:
# def remove_consideration(text):
    